In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
DATA_DIR: str = '/home/aaron/aaron_data/'
RAW_DF: str = os.path.join(DATA_DIR, "combined_documents.csv")
PATIENT_LEVEL_DF: str = os.path.join(DATA_DIR, "secondary_patient_level_annotations.csv")
PROBLEM_LEVEL_DF: str = os.path.join(DATA_DIR, "secondary_problem_level_annotations.csv")
REARRANGED_DATA_FILEPATH: str = os.path.join('/home/aaron/timeseries_nlp/data/', "rearranged_data.csv")
REARRANGED_DATA_FILEPATH_FLAT: str = os.path.join('/home/aaron/timeseries_nlp/data/', "rearranged_data_flat.csv")

In [2]:
for i in tqdm(range(0, 3), ncols=100, desc="Loading data.."):
    raw_df_as_read = pd.read_csv(RAW_DF)
    patient_level_annotations = pd.read_csv(PATIENT_LEVEL_DF)
    problem_level_annotations = pd.read_csv(PROBLEM_LEVEL_DF)
print("------Loading is completed ------")


Loading data..: 100%|█████████████████████████████████████████████████| 3/3 [00:35<00:00, 11.93s/it]

------Loading is completed ------


In [3]:
"Split the recorded date to DateTimeDay column, so we can get records existing on a single day"
raw_df = raw_df_as_read
raw_df[["DateTimeDay", "DateTimeSeconds"]] = raw_df["RecordedDate"].str.split(
    "T", expand=True
)
raw_df["Date"] = pd.to_datetime(raw_df["DateTimeDay"], format="%Y-%m-%d")
raw_df.drop(['Type', 'DateTimeSeconds'], axis=1, inplace=True)
raw_df = raw_df[(raw_df["Date"] >= "2019-1-1") & (raw_df["Date"] <= "2019-12-31")]


In [4]:
patient_level_annotations.head()


,PatientID,DataSiloName,CaseNumber,LatestPatientVersionID,BirthDate,Colour,Sex,IsNeutered,SexSource,BreedVeNomID,...,FirstVersionDate,LatestPatientVersionDate,FirstNoteDate,LastNoteDate,CodingStarted,FirstClinicId,SourceClinicName,SourceClinicPostcode,IsArchived,Is this patient included in the study_1
0,20718104,"Companion Care (Live,Rx)",19862UB,13077,07/04/2013 00:00:00,Black,Female,True,Female,14631.0,...,07/01/2020 18:02:01,01/15/2020 01:02:02,13/01/2020 14:05:56,13/01/2020 14:05:56,True,369.0,Hemel Hempstead Companion Care,NaN,False,Yes
1,12213778,"MediVet Group (Live,In-house)",119195341,17848,01/07/2013 00:00:00,Brown And Black,Male,False,Male,14880.0,...,15/02/2018 23:43:47,12/31/2021 05:19:50,17/12/2021 11:22:45,17/12/2021 11:22:45,True,1419.0,SHERWOOD,NG5 4AJ,False,Yes
2,8301118,"Companion Care (Live,Rx)",117580912Cm,14932,25/06/2011 00:00:00,Tan & White,Male,True,Male,15009.0,...,12/10/2017 00:22:17,09/06/2020 08:19:44,17/08/2020 11:17:56,17/08/2020 11:17:56,True,374.0,Cheltenham Companion Care,NaN,False,Yes
3,14874853,CVS,1213500010000074447,14969,07/01/2004 00:00:00,Black/white,Female,False,Female,NaN,...,23/03/2019 22:31:57,09/09/2020 09:27:18,08/10/2019 08:19:00,09/10/2019 16:21:00,True,1987.0,Roebuck Veterinary Centre,SG1 6DU,False,Yes
4,19035508,CVS,1213800010000120812,16512,17/05/2017 00:00:00,NaN,Male,False,Male,13962.0,...,NaN,04/13/2021 20:49:15,NaN,NaN,True,1992.0,Putlands Veterinary Surgery,TN12 6DZ,False,Yes


In [5]:

# Manual corrections to the data (Have already been updated on VC)
problem_level_annotations.at[627, "Is this note a visit_2"] = "Yes"
patient_level_annotations.at[897, "Is this patient included in the study_1"] = "Yes"
patient_level_annotations.at[206, "Is this patient included in the study_1"] = "Yes"
patient_level_annotations.at[925, "Is this patient included in the study_1"] = "Yes"
patient_level_annotations.at[969, "Is this patient included in the study_1"] = "Yes"
patient_level_annotations.at[173, "Is this patient included in the study_1"] = "Yes"


problem_level_annotations[["DateTimeDay", "DateTimeSeconds"]] = problem_level_annotations["DocumentDate"].str.split(
    " ", expand=True
)
problem_level_annotations["Date"] = pd.to_datetime(problem_level_annotations["DateTimeDay"], format="%d/%m/%Y")
coded_ehr = problem_level_annotations[problem_level_annotations['Is this note a visit_2'] == 'Yes']
problem_level_filter = list(coded_ehr[['PatientID', 'Date']].itertuples(index=False, name=None))
len(problem_level_filter)

3445

In [6]:
# lets get a list of all patients who are included in the study

included_patients_all_entries = patient_level_annotations.loc[
    patient_level_annotations["Is this patient included in the study_1"] == "Yes"
]
included_patients = list(set(included_patients_all_entries["PatientID"].tolist()))

# Sanity checking
if len(included_patients) != int(
    patient_level_annotations["Is this patient included in the study_1"].value_counts()[
        "Yes"
    ]
):
    raise ValueError(
        "There is a patient which has not been included which should have been!"
    )

print(f"We have {len(included_patients)} patients in this study")


We have 1214 patients in this study


In [7]:
# Lets remove all patients from the raw DF not included in the study
included_patients_ehr= raw_df[raw_df.PatientId.isin(included_patients)]

if len(set(included_patients) ^ set(included_patients_ehr.PatientId)):
    print(
        f"The following patients are not in both lists, please check they have no EHRs associated: {set(included_patients) ^ set(included_patients_ehr.PatientId)}"
    )

# Known patients with no EHRs - 16347576, 20619167
print(f"Total Patients with EHRs: {included_patients_ehr.PatientId.nunique()}")

The following patients are not in both lists, please check they have no EHRs associated: {16347576, 20619167}
Total Patients with EHRs: 1212


In [8]:
# lets filter the visits by if they are included
included_patients_ehr["included"] = np.nan
for index, row in included_patients_ehr.iterrows(): 
    check_ehr = (row.PatientId, row.Date)
    if check_ehr in problem_level_filter:
        included_patients_ehr.loc[index, 'included'] = 'Yes'
included_patients_ehr = included_patients_ehr.loc[included_patients_ehr['included'] == 'Yes']
included_patients_ehr

/tmp/ipykernel_26859/1718079723.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  included_patients_ehr["included"] = np.nan
/tmp/ipykernel_26859/1718079723.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  included_patients_ehr.loc[index, 'included'] = 'Yes'


,PatientId,RecordedDate,Document,DateTimeDay,Date,included
6467,17323493,2019-04-17T00:00:00,Nobivac Dhp 50D ; Vaccinate Dog - Booster ; No...,2019-04-17,2019-04-17,Yes
6468,17323493,2019-04-17T15:11:00,wt 10.3 kg \r\nDoing well for its age. Occasio...,2019-04-17,2019-04-17,Yes
6469,17323493,2019-05-17T00:00:00,Comprehensive Panel ; Consultation - Repeat ; ...,2019-05-17,2019-05-17,Yes
6470,17323493,2019-05-17T09:21:00,"Rex dog: Weight=9.8kg, lost some wt, wobbly on...",2019-05-17,2019-05-17,Yes
6471,17323493,2019-05-17T16:45:00,Req water intake measurement & u sample for di...,2019-05-17,2019-05-17,Yes
...,...,...,...,...,...,...
4361731,14161863,2019-08-01T08:52:00,"Improved from yesterday, no V+ in last 24 hour...",2019-08-01,2019-08-01,Yes
4361740,14161863,2019-12-12T00:00:00,Vaccinate & Health Check Dog - Booster ; Nobiv...,2019-12-12,2019-12-12,Yes
4361741,14161863,2019-12-12T11:38:00,Doing very well today. Pharmacy having trouble...,2019-12-12,2019-12-12,Yes
4366466,13811886,2019-04-17T14:06:33,"FA: Booster\r\n\r\nO has no concerns, eduf all...",2019-04-17,2019-04-17,Yes


In [9]:
df_new = pd.DataFrame(columns=['PatientId', 'EHR', 'Date', 'DOY', 'DOW'])

def day_of_week_num(dts):
    return (dts.astype('datetime64[D]').view('int64') - 4) % 7

import numpy as np

study_start_date = np.datetime64('2019-01-01')
index: int = 0
for patient_index in included_patients_ehr.PatientId.unique():
    data = included_patients_ehr[included_patients_ehr.PatientId == patient_index]
    for date in data.Date.unique():
        ehr = included_patients_ehr[(included_patients_ehr.Date == date) & (included_patients_ehr.PatientId == patient_index)]
        df_new.at[index, 'PatientId'] = patient_index
        df_new.at[index, 'EHR'] = " ".join(ehr.Document.values)
        df_new.at[index, 'Date'] = date
        df_new.at[index, 'DOW'] = day_of_week_num(date)
        df_new.at[index, 'DOY'] = pd.Timestamp(date).day_of_year
        index += 1

df_new

,PatientId,EHR,Date,DOY,DOW
0,17323493,Nobivac Dhp 50D ; Vaccinate Dog - Booster ; No...,2019-04-17T00:00:00.000000000,107,2
1,17323493,Comprehensive Panel ; Consultation - Repeat ; ...,2019-05-17T00:00:00.000000000,137,4
2,17323493,Consultation - Repeat wt = 9kgs r/c today as g...,2019-08-22T00:00:00.000000000,234,3
3,17323493,euthanasia dog <10kg ; Cremation Only Ashes NO...,2019-08-24T00:00:00.000000000,236,5
4,20517276,injection standard fee ; dispensing fee ; meta...,2019-09-23T00:00:00.000000000,266,0
...,...,...,...,...,...
3419,14161863,Fi Pre-anaesthetic Profile ; Lab in-hse MiLab ...,2019-06-06T00:00:00.000000000,157,3
3420,14161863,HPC 6 Month Check hpc check \r\n\r\nOR no heal...,2019-07-04T00:00:00.000000000,185,3
3421,14161863,Prevomax Inj 10mg/ml ; Dispensing Fee Tablets ...,2019-08-01T00:00:00.000000000,213,3
3422,14161863,Vaccinate & Health Check Dog - Booster ; Nobiv...,2019-12-12T00:00:00.000000000,346,3


In [10]:
doy = list(range(1, 365))  # Unsuprisingly, there are 365 days in a year
ts_df = pd.DataFrame(columns=doy)  # add 365 day of year columns to the new dataframe

print(len(df_new.PatientId.unique()))

for index in tqdm(df_new.PatientId.unique(), desc="Rearranging patient data"):
    patient_data = df_new[df_new.PatientId == index]
    for index, row in patient_data.iterrows():
        ts_df.at[row.PatientId, row.DOY] = row.EHR


1078


Rearranging patient data: 100%|██████████| 1078/1078 [00:02<00:00, 454.57it/s]


In [11]:
# Some ground truthes to ensure that its working well, patient 17323493 should have 4 visits 
if len(ts_df.loc[17323493][ts_df.loc[17323493].notnull()].values) != 4:
    raise ValueError('Issue with patient arrangement')


In [12]:
ts_df.to_csv(REARRANGED_DATA_FILEPATH)
# df_new.to_csv(REARRANGED_DATA_FILEPATH_FLAT)

: 